In [1]:
import requests
from shapely.geometry import LineString
import pandas as pd
import json
import osmnx as ox
import pyrosm
import folium

def find_edges(G, hwys):
    edges = []
    for u, v, k, data in G.edges(keys=True, data='highway'):
        check1 = isinstance(data, str) and data not in hwys
        check2 = isinstance(data, list) and all([d not in hwys for d in data])
        if check1 or check2:
            edges.append((u, v, k))
    return set(edges)

hwy_colors = {
        'motorway': 'purple',
        'trunk': 'pink',
        'primary': 'green',
        'secondary': 'yellow',
        'tertiary': 'orange',
        'unclassified': 'blue',
        'residential': 'red',
    }

In [51]:
# Create request URL
BASE_URL = "https://api.tomtom.com/"
API_KEY = "ThfURaOhGRaty7mx9AKALwrbyIZApHd8"
#zoom = 22
TRAFFIC_URL = "traffic/services/4/flowSegmentData/absolute/22/json?key="

UNIT = "unit=kmph"
ROAD_CLOSURE = "roadClosure=True"
THICKNESS = "thickness=1"

PARAMS = "&" + UNIT + "&" + ROAD_CLOSURE# + "&" + THICKNESS

In [52]:
ADDRESS = "Mayfair, London"
G = ox.graph_from_address(ADDRESS, network_type="drive", dist=400)

/Users/khalid-alsheeb/opt/anaconda3/envs/kurf/lib/python3.10/site-packages/osmnx/utils_geo.py:280: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
/Users/khalid-alsheeb/opt/anaconda3/envs/kurf/lib/python3.10/site-packages/osmnx/utils_geo.py:374: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:
/Users/khalid-alsheeb/opt/anaconda3/envs/kurf/lib/python3.10/site-packages/osmnx/utils_geo.py:374: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:


In [53]:
# first plot all edges that do not appear in hwy_colors's types
G_tmp = G.copy()
G_tmp.remove_edges_from(G.edges - find_edges(G, hwy_colors.keys()))
m = folium.Map()

# then plot each edge type in hwy_colors one at a time
for hwy, color in hwy_colors.items():
    G_tmp = G.copy()
    G_tmp.remove_edges_from(find_edges(G_tmp, [hwy]))
    if G_tmp.edges:
        m = ox.plot_graph_folium(   
                            G_tmp,
                            graph_map=m,
                            popup_attribute='name',
                            weight=5,
                            color=color
                        )

m

In [63]:
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
nodes, edges = ox.graph_to_gdfs(G)
coords = list(edges.iloc[2].geometry.coords)
correct_coords = []
for i in coords:
    correct_coords.append((i[1], i[0]))

In [64]:
latitude = str(correct_coords[5][0])
longitude = str(correct_coords[5][1])
location = "point=" + latitude + ","+ longitude

request_url = BASE_URL + TRAFFIC_URL + API_KEY + "&" + location + PARAMS
# Get data
response = requests.get(request_url)

In [65]:
data = response.json()['flowSegmentData']
traffic = pd.json_normalize(data)
traffic = traffic.rename(columns={'coordinates.coordinate': 'coordinates'})
traffic

,frc,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure,@version,coordinates
0,FRC7,20,20,150,150,1,False,traffic-service-flow 1.0.048,"[{'latitude': 51.51529573284709, 'longitude': ..."


In [66]:


coordinates = pd.json_normalize( traffic.iloc[0].coordinates)
#coordinates

#coordinates[round(coordinates.latitude, 7) == 34.0529563]
#coordinates.iloc[0].latitude
#coordinates.iloc[1]
coordinates.latitude = coordinates.latitude.round(7)
coordinates.longitude = coordinates.longitude.round(7)



In [67]:
geometry = LineString([(xy)  for xy in zip(coordinates['latitude'], coordinates['longitude'])])
#list(geometry.coords)


In [68]:
map = folium.Map(location=[correct_coords[0][0], correct_coords[0][1]],zoom_start=15)
folium.PolyLine(list(geometry.coords)).add_to(map)
folium.PolyLine(correct_coords).add_to(map)

In [69]:
for p in list(geometry.coords) + correct_coords:
    folium.Marker(p, popup=p).add_to(map)

In [70]:
map

In [ ]:
correct_coords